In [ ]:
# dpo
# from dataclasses import dataclass, field
from typing import Dict, Optional

# import torch
from datasets import Dataset, load_dataset
# from peft import LoraConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments

# from trl import DPOTrainer

def extract_anthropic_prompt(prompt_and_response, search_term="\n\nAssistant:"):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]


def get_hh(split: str, sanity_check: bool = False, silent: bool = False, cache_dir: str = None) -> Dataset:
    """Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts should be structured as follows:
      \n\nHuman: <prompt>\n\nAssistant:
    Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
    """
    dataset = load_dataset("Anthropic/hh-rlhf", split=split, cache_dir=cache_dir)
    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 1000)))

    def split_prompt_and_responses(sample) -> Dict[str, str]:
        prompt = extract_anthropic_prompt(sample["chosen"])
        return {
            "prompt": prompt,
            "chosen": sample["chosen"][len(prompt) :],
            "rejected": sample["rejected"][len(prompt) :],
        }

    return dataset.map(split_prompt_and_responses)

In [ ]:
split = 'train'
cache_dir = None
dataset = load_dataset("Anthropic/hh-rlhf", split=split, cache_dir=cache_dir)

In [2]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset

from fastchat.model.model_adapter import get_conversation_template

def extract_anthropic_prompt(prompt_and_response, search_term="\n\nAssistant:"):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]

class hankang_DPODataset:
    def __init__(
        self, 
        dataset_path="/data/llm_datasets/Ultrafeedback_binarized.ko.hankang/",
        data_format='chat-orca',
        search_term='\n\n### Assistant:',
        num_train=None,
        num_eval=None,
    ):
        self.dataset_path = dataset_path
        self.data_format = data_format
        self.search_term = search_term
        self.num_train = num_train
        self.num_eval = num_eval
    
    def get_prompt_and_response(self, data):
        conv = get_conversation_template(self.data_format)

        for idx, _conv in enumerate(data):
            role = _conv['role']
            content = _conv['content_kr']
            if idx % 2 == 0 and role == 'user':
                conv.append_message(conv.roles[0], content)
            elif idx % 2 == 1 and role == 'assistant':
                conv.append_message(conv.roles[1], content)
            else:
                print("Warning: data type invaild")

        if len(conv.messages) == 0:
            print("Warning: data is empty")
        if len(conv.messages) % 2 != 0:
            print("Warning: data has weird pair")

        return conv.get_prompt()
    
    def make_dpo_data_module(self):
        def validate_prompt_and_responses(data) -> bool:
            try:
                prompt_and_response = self.get_prompt_and_response(data['chosen'])
                prompt_and_response_rejected = self.get_prompt_and_response(data['rejected'])
                prompt = extract_anthropic_prompt(prompt_and_response, self.search_term)
                promopt_rejected = extract_anthropic_prompt(prompt_and_response_rejected, self.search_term)
            except AssertionError:
                return False

            return True

        def split_prompt_and_responses(data) -> Dict[str, str]:
            prompt_and_response = self.get_prompt_and_response(data['chosen'])
            prompt_and_response_rejected = self.get_prompt_and_response(data['rejected'])
            prompt = extract_anthropic_prompt(prompt_and_response, self.search_term)
            promopt_rejected = extract_anthropic_prompt(prompt_and_response_rejected, self.search_term)
            return {
                "prompt": prompt,
                "chosen": prompt_and_response[len(prompt) :],
                "rejected": prompt_and_response_rejected[len(promopt_rejected) :],
            }
                             
                             
        dataset = load_dataset(self.dataset_path)

        train_dataset = dataset['train']
        eval_dataset = dataset['test']

        original_columns = list(train_dataset.features.keys())

        if self.num_train is not None:
            train_dataset = train_dataset.select(range(min(len(train_dataset), self.num_train)))
        if self.num_eval is not None:
            eval_dataset = eval_dataset.select(range(min(len(train_dataset), self.num_eval)))

        train_dataset = train_dataset.filter(validate_prompt_and_responses)
        train_dataset = train_dataset.map(split_prompt_and_responses, remove_columns=original_columns)

        eval_dataset = eval_dataset.filter(validate_prompt_and_responses)
        eval_dataset = eval_dataset.map(split_prompt_and_responses, remove_columns=original_columns)

        return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

In [1]:
from fastchat.train.data_modules.dpo_dataset import hankang_DPODataset

dpo_dataset = hankang_DPODataset()
dpo_datamodule = dpo_dataset.make_dpo_data_module()

/home/ados/anaconda3/envs/fastchat_train_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eval_dataset = dpo_datamodule['eval_dataset']

In [ ]:
eval_dataset[0]

In [ ]:
from fastchat.conversation import (
    SeparatorStyle,
)
conv = get_conversation_template('chat-orca')
system_message = conv.system_message
sep_style = conv.sep_style
sep = conv.sep
prompt_user, prompt_bot = conv.roles

len_sep_style = 0
if sep_style == SeparatorStyle.ADD_COLON_TWO:
    len_sep_style = 1

len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
len_rear = len(sep) + len(prompt_bot) + len_sep_style




In [10]:
import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
import copy

def dedup_dpo_datamodule(dpo_datamodule, target_text_len=100, n_results=100, threshold = 0.6):
    # prepare datamodule
    train_dataset = dpo_datamodule['train_dataset']
    eval_dataset = dpo_datamodule['eval_dataset']
    

    
    
    conv = get_conversation_template('chat-orca')
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

    train_dataset = train_dataset.map(filter_question)
    eval_dataset = eval_dataset.map(filter_question)
    
    for _data_id, dataset in enumerate([train_dataset, eval_dataset]):
        chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
        embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
        collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
        ids = []
        # add
        texts = dataset['prompt']
        last_id = -1
        new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
        ids += new_ids
        collection.add(documents=texts, ids=new_ids)
        
        query_ids = copy.deepcopy(new_ids)
        selected_ids = []
        duplicated_ids = []
        
        while query_ids:
            current_id = random.choice(query_ids)
            selected_ids.append(current_id)
            search_strings = [texts[int(current_id[2:])]]
            result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

            search_ids = result['ids'][0]
            distances = result['distances'][0]
            remove_ids = []
            for idx in range(len(search_ids)):
                sid = search_ids[idx]
                dist = distances[idx]
                if dist < threshold:
                    remove_ids.append(sid)

            for rid in remove_ids:
                if rid in query_ids:
                    query_ids.remove(rid)
            duplicated_ids += remove_ids
            collection.delete(ids=remove_ids)

            print(len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids), '\t\t\t\t\t', end='\r')
        
        if _data_id == 0:
            print('finished dedup train data:', len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids))
        else:
            print('finished dedup eval data:', len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids))

In [3]:
import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder

In [4]:
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []

In [5]:
len_system = len('### System:\nThis is a system prompt, please behave and help the user.')
len_user = len('\n\n### User: ')
len_bot = len('\n\n\n### Assistant:')
text_len = 100
def remove_system(data):
    return { 
        **data,
        'prompt': data['prompt'][len_system + len_user:-len_bot][:text_len]
    }

eval_dataset = eval_dataset.map(remove_system)

In [6]:
eval_dataset[30]

{'rejected': ' 히비스쿠스 빌라는 산림 관리 목재, 풍성한 야자 나무 기둥, 알랑알랑 풀 초가지 지붕, 고급 물 양치류 강 모시 직물 및 원목 가구와 천연 석벽과 수생 동물을 포함한 천연 소재를 사용하여 지어졌습니다. 이 빌라는 보르네오 해안의 멋진 위치에 자리 잡고 있으며 백사장과 개인 동굴, 맑은 바다와 야자수 등 짧은 산책 거리 내에 있습니다. 개인 집사가 24시간 서비스를 제공하며 숙박 기간 동안 모든 필요를 충족시켜 드립니다.\n\n이 빌라는 2에이커 이상의 개인 열대 정원이 있으며, 대형 개인 천연 대리석 수영장이 있어 공간과 프라이버시를 보장하고 고립된 느낌을 줍니다. 게스트는 매일 청소 서비스, 세탁 서비스, 완비된 주방, 수영장, 패들 풀, 열대 정원 및 선 베드에서 이용할 수 있습니다. 또한 이 빌라에서는 무료 Wi-Fi 접속과 홈 시네마 및 게임 룸을 포함한 다양한 엔터테인먼트 옵션도 제공합니다.\n\n히비스쿠스 빌라는 멋진 자연 환경에서 고급스럽고 친밀한 휴식 경험을 원하는 분들에게 완벽한 선택입니다. 풀장에서 휴식을 취하거나 해변에서 일광욕을 즐기거나 현지 지역을 탐험하고 싶으시다면 빌라의 위치와 시설이 보르네오 여행의 이상적인 베이스가 될 것입니다.\n</s>',
 'prompt': '히비스쿠스 빌라의 건축에 어떤 자재가 사용되며, 게스트가 이용할 수 있는 시설과 서비스는 무엇인가요?\n에르세테는 다음과 같이 제공됩니다: 히비스쿠스 빌라는 보르네오 해안의 뛰어난 ',
 'chosen': ' 히비스쿠스 빌라는 보르네오 해안에 위치한 고급스럽고 프라이빗한 빌라입니다. 이 빌라는 개인 집사가 서비스를 제공하며 2에이커 이상의 개인 열대 정원이 있습니다. 이 숙소는 아름다운 정원을 갖추고 있으며, 대형 천연 암반 수영장이 공간과 프라이버시를 제공합니다. 이 빌라는 숲을 가꾸는 원목, 풍성한 야자수 기둥, 알랑알랑 풀 초가지붕 지붕, 고급 수생 양치류 강모초 직조 및 목재 가구와 자연 석벽 및 물 기능을 포함한 천연 소재로 지어졌습니

In [7]:
# add
texts = eval_dataset['prompt']

last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

In [81]:
eval_dataset['prompt'][0][:]

'### User: 스페인어 문장이 주어집니다. 여러분의 임무는 스페인어 문장을 갈리시아어로 번역하는 것입니다.\n\n입력:\n아니요, 저는 완벽한 평등에 대해 말하고 있지 않습니다. 민주주의가 발달한 부유한 시장에서 존재하는 것에 대해 이야기하고 있습니다.\n\n출력:\n저는 완벽에 가까운 말을 하고 있습니다. 부유한 시장에서 존재하는 것에 대해 말합니다.\n\n\n입력:\n늑대와 비슷한 생명체부터 시작하여 말테스로 마무리합니다.\n\n출력:\n늑대처럼 생긴 생물을 만들고 몰트의 맛을 더했습니다.\n\n\n입력:\n그래서 이것은 해초 과일입니다. 이렇게 불립니다.\n\n출력:\n\n\n\n### Assistant:'

In [8]:
len(new_ids)

1999

In [9]:
import copy
query_ids = copy.deepcopy(new_ids)

n_results = 100
threshold = 0.6

selected_ids = []
duplicated_ids = []
# while query_ids:

In [10]:
while query_ids:
    
    current_id = random.choice(query_ids)
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < threshold:
            remove_ids.append(sid)
    
    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
    duplicated_ids += remove_ids
    collection.delete(ids=remove_ids)
    
    print(len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids), '\t\t\t\t\t', end='\r')
    

In [378]:
selected_ids[-3:]

['id53806', 'id44222', 'id42888']

In [379]:
print(len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids), '\t\t\t\t\t', end='\r')

In [385]:
len(set(selected_ids))

61993

In [382]:
texts[2]

'사람들이 유니폼을 입은 축구 선수가 모래밭에서 발을 차는 모습을 지켜봅니다.\n질문과 답변은 아래에 있습니다.\n"검은 옷을 입은 검은 머리의 소년이 모래밭에서 축구공을 찬다."라는 '

In [11]:
selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

eval_dataset = dpo_datamodule['eval_dataset']
eval_dataset = eval_dataset.select(selected_ids)

In [16]:
eval_dataset

Dataset({
    features: ['rejected', 'prompt', 'chosen'],
    num_rows: 877
})

In [358]:
len(query_ids), collection.count()

(36750, 36750)

In [13]:
eval_dataset.to_json(f"dpo-eval.jsonl", )

Creating json from Arrow format: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.93ba/s]


5736347

In [335]:
10057

['id0']

In [332]:
remove_ids= 

In [336]:
for _ in range(5):
    collection.delete(ids=new_ids[1:])

In [337]:
result = collection.query(query_texts=search_strings, n_results=n_results, include=['distances'])

Number of requested results 100 is greater than number of elements in index 1, updating n_results = 1


In [338]:
result = collection.query(query_texts='먼저 작업에 대한 정의가 주어지고, 그 다음에 작업의 입력이 주어집니다.\nGujarati로 된 텍스트가 주어집니다. 구자라트어에서 파잔비어로 번역합니다. 번역은 원문 문장에 정보를', n_results=100, include=['documents', 'distances'])

Number of requested results 100 is greater than number of elements in index 1, updating n_results = 1


In [184]:
%%time
collection.delete(ids=remove_ids)

CPU times: user 126 ms, sys: 195 ms, total: 321 ms
Wall time: 381 ms


In [189]:
'id26896' in remove_ids

False

In [339]:
result['documents'][0], result['distances'][0]

(['스페인어 문장이 주어집니다. 여러분의 임무는 스페인어 문장을 갈리시아어로 번역하는 것입니다.\n\n입력:\n아니요, 저는 완벽한 평등에 대해 말하고 있지 않습니다. 민주주의가 발달한 부'],
 [0.7323936223983765])

In [ ]:
result['ids'][0]

In [172]:
search_strings

['먼저 작업에 대한 정의가 주어지고, 그 다음에 작업의 입력이 주어집니다.\nGujarati로 된 텍스트가 주어집니다. 구자라트어에서 파잔비어로 번역합니다. 번역은 원문 문장에 정보를']

In [ ]:
from trl import DPOTrainer
from transformers import Trainer

In [ ]:
DPOTrainer(

In [ ]:
import transformers
dir(transformers.TrainingArguments)

In [3]:
a = 853/46302 
#16:08:08

In [4]:
x = 16/ (a)

In [6]:
159182/60/60

44.21722222222222

In [1]:
from datasets import load_dataset
dataset = load_dataset('dpo_sample')

/home/ados/anaconda3/envs/fastchat_train_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['rejected', 'prompt', 'chosen'],
        num_rows: 10316
    })
    test: Dataset({
        features: ['rejected', 'prompt', 'chosen'],
        num_rows: 877
    })
})

In [4]:
dataset['train'][2]

{'rejected': ' 물론이죠! 이것은 흥미로운 개념입니다. 다음은 창의적인 프로세스를 통해 새로운 커피 아이디어를 창출하는 동시에 흥미롭고 재미있게 만드는 데 도움이 될 수 있는 창의적인 프로세스입니다:\n\n1. 커피 테마 브레인스토밍 세션:\n커피 애호가부터 요리 전문가에 이르기까지 다양한 배경의 개인들로 그룹을 구성하세요. 이를 통해 커피 아이디어에 대한 폭넓은 관점을 확보할 수 있습니다.\n2. 아이스 브레이커:\n모두에게 좋아하는 커피 이야기나 특별한 커피와 관련된 기억을 공유하도록 요청하여 세션을 시작하세요. 창의적인 분위기를 조성하는 재미있는 방법입니다.\n3. 그룹 활동:\n참가자들을 소그룹으로 나누고 기본 커피 재료(커피 원두, 우유, 설탕 등)와 일반적인 커피 추출 방법(에스프레소, 카푸치노, 라테 등) 목록을 제공합니다.\n4. 창의적 마인드 매핑:\n각 그룹은 각 재료 또는 방법에 대한 다양한 커피 아이디어와 레시피로 마인드 맵을 생성해야 합니다. 각 그룹 구성원이 자유롭게 아이디어를 공유하고 마인드 맵에 적극적으로 참여하도록 장려하세요.\n5. 그룹 프레젠테이션:\n정의된 시간이 지나면 각 그룹이 자신의 아이디어를 더 큰 그룹에 발표합니다. 이렇게 하면 모든 사람이 생성된 다양한 아이디어를 보고 대화와 창의성을 자극하는 데 도움이 됩니다.\n6. 그룹 협업:\n각 그룹은 각 그룹에서 가장 좋아하는 아이디어 세 가지를 선택하고, 가장 인기 있고 혁신적인 아이디어라고 생각하는 상위 아이디어에 투표해야 합니다. 이렇게 하면 모든 그룹의 관심사를 바탕으로 최선의 아이디어가 선정되고 정당화되며 뒷받침됩니다.\n7. 실험 및 개선:\n각 그룹이 레시피와 준비 과정에 모두 기여하여 우승 아이디어를 더욱 발전시킬 수 있습니다. 각 그룹은 음료를 만들어 맛에 대한 피드백을 제공하고 완벽한 레시피를 완성할 때까지 다듬을 수 있습니다.\n8. 최종 시음 및 선택:\n최종 제품을 모두 맛보고 어떤 맛을 좋아하는지 확인하세요. 이를 통해 최종 제품의 개